In [ ]:
# %load bertFt.py
"""
    per tweet
"""

import re
import pandas as pd
import torch
import os

from torch import nn
from transformers import BertTokenizer, BertModel
from myDataSet import MyTokenizerDataset
from torch.utils.data import DataLoader
from fit import train, test
import utils

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)


learning_rate = 1e-2
split_ratio=0.2
input_size = 768
epochs = 10
batch_size = 1024
decay_rate = 0.95


class Net(nn.Module):
    # since only two classes in train data
    # use sigmoid to classify
    def __init__(self, inSize, classes=1):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(inSize, int(inSize/2), 2, batch_first=True,bidirectional=False)
        self.dropout = nn.Dropout(0.5)
#         self.fc1 = nn.Linear(inSize, 256)
#         self.act1 = nn.ReLU()
#         self.fc2 = nn.Linear(256, 64)
#         self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(int(inSize/2), classes)
        if classes < 2:
            self.classify = nn.Sigmoid()
        else:
            self.classify = nn.Softmax(dim=1)

    def forward(self, x,hidden):
        batch_size = x.size(0)
#         x=x.view(len(x), 1, -1) 
        lstm_out, (hidden_last,cn_last) = self.lstm(x, hidden)
        hidden_last_out=hidden_last[-1]
#         out = self.fc1(x)
#         out = self.act1(out)
#         out = self.fc2(out)
#         out = self.act2(out)
        out = self.fc3(hidden_last_out)
        out = self.classify(out)
        return out
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        number = 1

        hidden = (weight.new(2*number, batch_size, 384).zero_().float().cuda(),
                  weight.new(2*number, batch_size, 384).zero_().float().cuda()
                 )

        
        return hidden

    
trainPath = './data/traintweetswithlabel_1.csv'
train_csv=pd.read_csv(trainPath)
dfTrain=pd.DataFrame(train_csv)
dfTrainauc=dfTrain.loc[dfTrain['label']==1]
dfTrain=pd.concat([dfTrain,dfTrainauc], ignore_index=True)
train_counts=pd.value_counts(dfTrain['label']).tolist()
print(train_counts)
train_data=MyTokenizerDataset(dfTrain)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True,drop_last=True)

# train_counts=[pd.value_counts(dfTrain['label'])[0],pd.value_counts(dfTrain['label'])[1]]
# train_counts=pd.value_counts(dfTrain['label']).tolist()
# print(train_counts)
# train_weights= 1./ torch.tensor(train_counts, dtype=torch.float)
# train_targets = train_data.getLabels()
# train_samples_weights = train_weights[train_targets]
# train_sampler = torch.utils.data.WeightedRandomSampler(weights=train_samples_weights, num_samples=len(train_samples_weights), replacement=True)
# train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, sampler=train_sampler, num_workers=4,pin_memory=True)


testPath = './data/testtweetswithlabel_1.csv'
test_csv=pd.read_csv(testPath)
dfTest=pd.DataFrame(test_csv)
test_data = MyTokenizerDataset(dfTest)

test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True,drop_last=True)


# test_counts=[pd.value_counts(dfTest['label'])[0],pd.value_counts(dfTest['label'])[1]]
# test_counts=pd.value_counts(dfTest['label']).tolist()
# print(test_counts)
# test_weights= 1./ torch.tensor(test_counts, dtype=torch.float)
# test_targets = test_data.getLabels()
# test_samples_weights = test_weights[test_targets]
# test_sampler = torch.utils.data.WeightedRandomSampler(weights=test_samples_weights, num_samples=len(test_samples_weights), replacement=True)
# test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, sampler=test_sampler, num_workers=4,pin_memory=True)

# test_size=int(len(data)*split_ratio)
# train_size=len(data)-test_size
# train_data, test_data = torch.utils.data.random_split(data, [train_size, test_size])


# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained("bert-base-uncased").to(device)
num_features = bert.pooler.dense.in_features

net = Net(num_features, 1).to(device)
criterion = nn.BCELoss()
# criterion = utils.BCEFocalLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay_rate)
# train(net, train_loader, test_loader, bert, tokenizer, optimizer, criterion, scheduler, device, epochs)
# test(net, test_loader, bert, tokenizer, criterion, device)
test(net,test_loader,bert,criterion,device,batch_size)
train(net, train_loader, test_loader, bert, optimizer, criterion, scheduler, device, epochs,batch_size)
test(net,test_loader,bert,criterion,device,batch_size)


In [ ]:
# %load bertFt.py
"""
    per user
"""

import re
import pandas as pd
import torch
import os

from torch import nn
from transformers import BertTokenizer, BertModel
from myDataSet import MyTokenizerDataset
from torch.utils.data import DataLoader
from fit import train, test
import utils

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)


learning_rate = 1e-7
split_ratio=0.2
input_size = 768
epochs = 100
batch_size = 64
decay_rate = 0.95


class Net(nn.Module):
    # since only two classes in train data
    # use sigmoid to classify
    def __init__(self, inSize, classes=1):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(inSize, int(inSize/2), 2, batch_first=True,bidirectional=False)
        self.dropout = nn.Dropout(0.5)
#         self.fc1 = nn.Linear(inSize, 256)
#         self.act1 = nn.ReLU()
#         self.fc2 = nn.Linear(256, 64)
#         self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(int(inSize/2), classes)
        if classes < 2:
            self.classify = nn.Sigmoid()
        else:
            self.classify = nn.Softmax(dim=1)

    def forward(self, x,hidden):
        batch_size = x.size(0)
#         x=x.view(len(x), 1, -1) 
        lstm_out, (hidden_last,cn_last) = self.lstm(x, hidden)
        hidden_last_out=hidden_last[-1]
#         out = self.fc1(x)
#         out = self.act1(out)
#         out = self.fc2(out)
#         out = self.act2(out)
        out = self.fc3(hidden_last_out)
        out = self.classify(out)
        return out
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        number = 1
#         if self.bidirectional:
#             number = 2
        

        hidden = (weight.new(2*number, batch_size, 384).zero_().float().cuda(),
                  weight.new(2*number, batch_size, 384).zero_().float().cuda()
                 )

        
        return hidden

    
trainPath = './data/mergetraintweetswithlabel_1.csv'
train_csv=pd.read_csv(trainPath)
dfTrain=pd.DataFrame(train_csv)

dfTrainauc=dfTrain.loc[dfTrain['label']==1]
dfTrain=pd.concat([dfTrain,dfTrainauc], ignore_index=True)
dfTrain=dfTrain.sample(frac=1).reset_index(drop=True)
train_counts=pd.value_counts(dfTrain['label']).tolist()
print(train_counts)
train_data=MyTokenizerDataset(dfTrain)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True,drop_last=True)

# train_counts=[pd.value_counts(dfTrain['label'])[0],pd.value_counts(dfTrain['label'])[1]]
# train_counts=pd.value_counts(dfTrain['label']).tolist()
# print(train_counts)
# train_weights= 1./ torch.tensor(train_counts, dtype=torch.float)
# train_targets = train_data.getLabels()
# train_samples_weights = train_weights[train_targets]
# train_sampler = torch.utils.data.WeightedRandomSampler(weights=train_samples_weights, num_samples=len(train_samples_weights), replacement=True)
# train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, sampler=train_sampler, num_workers=4,pin_memory=True)


testPath = './data/mergetesttweetswithlabel_1.csv'
test_csv=pd.read_csv(testPath)
dfTest=pd.DataFrame(test_csv)
test_data = MyTokenizerDataset(dfTest)

test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True,drop_last=True)


# test_counts=[pd.value_counts(dfTest['label'])[0],pd.value_counts(dfTest['label'])[1]]
# test_counts=pd.value_counts(dfTest['label']).tolist()
# print(test_counts)
# test_weights= 1./ torch.tensor(test_counts, dtype=torch.float)
# test_targets = test_data.getLabels()
# test_samples_weights = test_weights[test_targets]
# test_sampler = torch.utils.data.WeightedRandomSampler(weights=test_samples_weights, num_samples=len(test_samples_weights), replacement=True)
# test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, sampler=test_sampler, num_workers=4,pin_memory=True)

# test_size=int(len(data)*split_ratio)
# train_size=len(data)-test_size
# train_data, test_data = torch.utils.data.random_split(data, [train_size, test_size])


# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained("bert-base-uncased").to(device)
num_features = bert.pooler.dense.in_features

net = Net(num_features, 1).to(device)
criterion = nn.BCELoss()
# criterion = utils.BCEFocalLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay_rate)
# train(net, train_loader, test_loader, bert, tokenizer, optimizer, criterion, scheduler, device, epochs)
# test(net, test_loader, bert, tokenizer, criterion, device)
test(net,test_loader,bert,criterion,device,batch_size)
train(net, train_loader, test_loader, bert, optimizer, criterion, scheduler, device, epochs,batch_size)
test(net,test_loader,bert,criterion,device,batch_size)
